In [1]:
import pandas as pd
import numpy as np
import re
import pickle

!pip install git+https://github.com/alberanid/imdbpy #imdbpy 설치
from imdb import IMDb

  Cloning https://github.com/alberanid/imdbpy to /tmp/pip-req-build-a5pfrod3
  Running command git clone -q https://github.com/alberanid/imdbpy /tmp/pip-req-build-a5pfrod3
  Created wheel for IMDbPY: filename=IMDbPY-2020.7.29-cp36-none-any.whl size=300835 sha256=6c922545831b84a8c6027afb6e912e681092f8c044066c16058ced5cef7262ea
  Stored in directory: /tmp/pip-ephem-wheel-cache-b3ri5cnh/wheels/0f/09/61/190df5e0276765680540f1562f2abca80e725a7e48595e993f
Successfully built IMDbPY


In [2]:
#구글드라이브 마운트
from google.colab import drive
drive.mount('/gdrive')
%cd /gdrive

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /gdrive
/gdrive


In [3]:
#새 경로에서 불러오기

#1. 구글 드라이브에서 Movie_data 다운로드, 합치기 
df = pd.DataFrame()
df_list = []
[df_list.append("df"+str(df_no)) for df_no in range(26)]
print('df_list:',df_list)
df = pd.DataFrame(data = None) #첫번째 df. 빔. 빈 df 만들기: pd.DataFrame(data = None)

for data_ind, df_element in zip(range(7),df_list): #**추후에 바꿔 range(26)으로 #ZIP 함수를 활용해서 for i,j in zip(--,--): 이렇게 변수 두개 쓸 수도 있음. 
  #박사님's path = "/gdrive/My Drive/movie/"+"movie_data_"+str(data_ind)+".csv"
  path = "/gdrive/My Drive/movie_predict/영화중복실험/movie_data_"+str(data_ind)+".csv"
  print("path:",path)
  print(df_element)


  df_element = pd.read_csv(path, index_col = "Movie_title") #pd.read_csv('폴더경로',index_col = "column이름") 
  print(df_element)
  df = pd.concat([df, df_element]) #df0,df1,df2,...


#df.describe()
df
  #이렇게 하면 df_list의 값에 바로 pd가 지정되는게 아니라 ex) 'df_list[data_ind]'가 pd로 지정됨. 각각 list안에값을 변수 이름으로 하고프면



df_list: ['df0', 'df1', 'df2', 'df3', 'df4', 'df5', 'df6', 'df7', 'df8', 'df9', 'df10', 'df11', 'df12', 'df13', 'df14', 'df15', 'df16', 'df17', 'df18', 'df19', 'df20', 'df21', 'df22', 'df23', 'df24', 'df25']
path: /gdrive/My Drive/movie_predict/영화중복실험/movie_data_0.csv
df0
                               Unnamed: 0  ...               Domestic_distributor
Movie_title                                ...                                   
The End of the Affair (1999)            0  ...  Sony Pictures Entertainment (SPE)
The Cider House Rules (1999)            1  ...                            Miramax
Magnolia (1999)                         2  ...                    New Line Cinema
Girl, Interrupted (1999)                3  ...  Sony Pictures Entertainment (SPE)
Snow Falling on Cedars (1999)           4  ...                 Universal Pictures
...                                   ...  ...                                ...
Fast Food Fast Women (2000)           495  ...              

,Unnamed: 0,Genre,Rate,Rating_count,Run_time,Budget,Gross_USA,Opening Weekend USA,Cumulative Worldwide Gross,Language,Directors,Writers,Stars,Certificate,Released_date,Domestic_distributor
Movie_title,,,,,,,,,,,,,,,,
The End of the Affair (1999),0,"['Drama', 'Romance']",['7.1'],"20,975",['1h 42min'],"$23,000,000 (estimated)","$10,827,816","$198,535, 5 December 1999","$10,827,816",['English'],['Neil Jordan'],"['Graham Greene', 'Neil Jordan']","['Ralph Fiennes', 'Stephen Rea', 'Julianne Moo...",['R'],21 January 2000,Sony Pictures Entertainment (SPE)
The Cider House Rules (1999),1,"['Drama', 'Romance']",['7.4'],"91,652",['2h 6min'],"$24,000,000 (estimated)","$57,545,092","$110,098, 12 December 1999","$88,545,092",['English'],['Lasse Hallstr'],"['John Irving', 'John Irving']","['Tobey Maguire', 'Charlize Theron', 'Delroy L...","['TV-PG', 'PG-13']",7 January 2000,Miramax
Magnolia (1999),2,['Drama'],['8.0'],"283,965",['3h 8min'],"$37,000,000 (estimated)","$22,455,976","$193,604, 19 December 1999","$48,451,803","['English ', ' German ', ' French']",['Paul Thomas Anderson'],['Paul Thomas Anderson'],"['Pat Healy ...', 'Genevieve Zweig', 'Mark Fla...",['R'],7 January 2000,New Line Cinema
"Girl, Interrupted (1999)",3,"['Biography', 'Drama']",['7.3'],"159,704",['2h 7min'],"$40,000,000 (estimated)","$28,912,646","$95,399, 26 December 1999","$48,350,205",['English'],['James Mangold'],"['Susanna Kaysen', 'James Mangold', 'Lisa Loom...","['Winona Ryder', 'Angelina Jolie', 'Clea DuVal...",['R'],14 January 2000,Sony Pictures Entertainment (SPE)
Snow Falling on Cedars (1999),4,"['Drama', 'Mystery', 'Romance']",['6.7'],"13,528",['2h 7min'],"$35,000,000 (estimated)","$14,417,593","$32,135, 26 December 1999","$23,049,593","['English ', ' Japanese ', ' German']",['Scott Hicks'],"['David Guterson', 'Ronald Bass', 'Scott Hicks']","['Ethan Hawke', 'Ethan Hawke ... Ishmael Chamb...",['PG-13'],7 January 2000,Universal Pictures
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
The Abandoned (2006),495,"['Horror', 'Mystery', 'Thriller']",['5.6'],"12,325",['1h 39min'],"EUR3,000,000 (estimated)","$1,653,012","$2,482,000, 19 November 2006","$4,529,892","['English ', ' Russian']",['Nacho Cerd'],"['Karim Hussain', 'Nacho Cerdà', 'Richard Stan...","['Anastasia Hille', 'Karel Roden', 'Valentin G...",['R'],23 February 2007,Freestyle Releasing
Amazing Grace (2006),496,"['Biography', 'Drama', 'History']",['7.4'],"23,610",['1h 58min'],NaN,"$21,250,683","$4,305,000, 25 February 2007","$32,120,360",['English'],['Michael Apted'],['Steven Knight'],"['Ioan Gruffudd', 'Romola Garai', 'Benedict Cu...",['PG'],23 February 2007,IDP Distribution
Starter for 10 (2006),497,"['Comedy', 'Drama', 'Romance']",['6.7'],"21,694",['1h 32min'],NaN,"$216,839","$37,141, 25 February 2007","$1,736,394",['English'],['Tom Vaughan'],"['David Nicholls', 'David Nicholls']","['Joseph Friend', 'James Gaddas', 'Catherine T...",['PG-13'],9 March 2007,Picturehouse


In [ ]:
df[df.index.duplicated()]
#2470 rows × 16 columns 안에는 중복 없었음. (movie_data0-4)
#movie_data_5,6에서 NaN문제
#movie_code도 같이 수집하자. webpage랑.

### 영화 수상 여부 확인

In [ ]:
#Stars 영화 수상 여부 확인
from ast import literal_eval #str에서 list를 인식할 수 있게 해줌. 

#노미니,위너 빈 컬럼 생성
df["nominated_stars"] = 0 
df["winner_stars"] = 0 
df["nominated_writers"] = 0 
df["winner_writers"] = 0 
df["nominated_directors"] = 0
df["winner_directors"] = 0

for n in range(len(df.index)): #영화접근
  #우선 ' ... ' cleansing 
  star_list = literal_eval(df['Stars'].iloc[n]) #n번째 행(영화) 행자리임 바꾸삼
  for i in range(len(star_list)): # n번째 영화안의 i번째 배우    
    #정규 표현식으로 각 배우' ... ' 없애기.
    try:
      m = re.search(r'(.*)\.\.\.(.*)',star_list[i]) # ( ... )는 띄어쓰기+ 모든문자(점 포함) 세번+띄어쓰기 매칭하는거.
      star = m.group(1)

    except: #이미 정상이어서 정규표현식이 안통하는 애들
      star = star_list[i]

    #Star 한명의 수상이력
    ia = IMDb()
    Person_Id = ia.search_person(star)[0].personID #검색되는 0번째 star check
    Awards = ['Golden Globes, USA', 'Academy Awards, USA']

    #len(ia.get_person_awards(Person_Id).get('data').get('awards'))
    if ia.get_person_awards(Person_Id).get('data').get('awards') == None:
      continue #수상 정보 없으므로 다음 배우로 
    else:
      award_records_no = len(ia.get_person_awards(Person_Id).get('data').get('awards'))
      award_records = ia.get_person_awards(Person_Id).get('data').get('awards')
      #print(award_records)

      df['Released_date'] = pd.to_datetime(df['Released_date']) #df['컬럼명'] = pd.to_datetime(df['컬럼명'])
      agg_award_year = df['Released_date'].dt.year[0] - 1 #n번째 영화니까  #agg_award_year = df['Released_date'].dt.year[n] - 1 #n번째 영화니까 ** else 다하고 바꾸기

      for record_no in range(award_records_no): #한 영화 안에서 배우한명의 record
        try:
          #print(award_records[record_no])
          awarded_year = award_records[record_no]['year'] #수상 연도
          right_year = awarded_year <= agg_award_year
          right_award = award_records[record_no]['award'] in Awards 
          if right_year and right_award : #연도 조건 충족 시 (조건1), 조건2 - 골든 글로브/acdemy award수상하는지
            #print(award_records[record_no])
            #노미니인지 승자인지?
            if ia.get_person_awards(Person_Id).get('data').get('awards')[0]['result'] == 'Nominee': #Nominee인 경우 Nominee칼럼에 1
              df["nominated_stars"].iloc[n] = 1  
            elif :ia.get_person_awards(Person_Id).get('data').get('awards')[0]['result'] == 'Winner':
              df["winner_stars"].iloc[n] = 1   

          else:
          continue #다음 이력

        except KeyError: #year가 key로 없으면 영화 제목에서 뽑아 쓰기 **안됨 일단 패스. 여기 꼭 고치셈 -->Ryan gosling으로 해보셈
          continue #print(award_records[record_no]['movies'])
        #except:
          #continue


NameError: ignored

In [21]:
#위에 코드 else 밑
ia = IMDb()
Person_Id = ia.search_person('leonardo dicaprio')[0].personID #Person_Id = ia.search_person(star)[0].personID else 다 하고 이코드로 바꾸셈 **

Awards = ['Golden Globes, USA', 'Academy Awards, USA']

#여기부터 붙여넣어 밑에 셀에
award_records_no = len(ia.get_person_awards(Person_Id).get('data').get('awards'))
award_records = ia.get_person_awards(Person_Id).get('data').get('awards')
#print(award_records)

df['Released_date'] = pd.to_datetime(df['Released_date']) #df['컬럼명'] = pd.to_datetime(df['컬럼명'])
agg_award_year = df['Released_date'].dt.year[0] - 1 #n번째 영화니까  #agg_award_year = df['Released_date'].dt.year[n] - 1 #n번째 영화니까 ** else 다하고 바꾸기

for record_no in range(award_records_no): #한 영화 안에서 배우한명의 record
  try:
    #print(award_records[record_no])
    awarded_year = award_records[record_no]['year'] #수상 연도
    right_year = awarded_year <= agg_award_year
    right_award = award_records[record_no]['award'] in Awards 
    if right_year and right_award : #연도 조건 충족 시 (조건1), 조건2 - 골든 글로브/acdemy award수상하는지
      #print(award_records[record_no])
      #노미니인지 승자인지?
      if ia.get_person_awards(Person_Id).get('data').get('awards')[0]['result'] == 'Nominee': #Nominee인 경우 Nominee칼럼에 1
        df["nominated_stars"].iloc[n] = 1  
      elif :ia.get_person_awards(Person_Id).get('data').get('awards')[0]['result'] == 'Winner':
        df["winner_stars"].iloc[n] = 1   

    else:
     continue #다음 이력

  except KeyError: #year가 key로 없으면 영화 제목에서 뽑아 쓰기 **안됨 일단 패스. 여기 꼭 고치셈 -->Ryan gosling으로 해보셈
    continue #print(award_records[record_no]['movies'])
  #except:
    #continue


#ia.get_person_awards(Person_Id).get('data').get('awards')[0] #수상 목록

#'movies': <Movie id:3783958[http] title:_La La Land (2016)_>


{'year': 1994, 'result': 'Nominee', 'prize': 'Oscar', 'category': 'Best Actor in a Supporting Role', 'movies': <Movie id:0108550[http] title:_What's Eating Gilbert Grape (1993)_>, 'award': 'Academy Awards, USA'}
{'year': 1998, 'result': 'Nominee', 'prize': 'Golden Globe', 'category': 'Best Performance by an Actor in a Motion Picture - Drama', 'movies': <Movie id:0120338[http] title:_Titanic (1997)_>, 'award': 'Golden Globes, USA'}
{'year': 1994, 'result': 'Nominee', 'prize': 'Golden Globe', 'category': 'Best Performance by an Actor in a Supporting Role in a Motion Picture', 'movies': <Movie id:0108550[http] title:_What's Eating Gilbert Grape (1993)_>, 'award': 'Golden Globes, USA'}


In [18]:
ia.search_person('leonardo dicaprio')[0]

<Person id:0000138[http] name:_Leonardo DiCaprio_>

In [23]:
df['Stars'].iloc[0]

'[\'Ralph Fiennes\', \'Stephen Rea\', \'Julianne Moore\', "Heather-Jay Jones ... Henry\'s Maid", \'James Bolam ...\', \'Ian Hart ...\', \'Sam Bould ... Lance Parkis\', \'Cyril Shaps\', \'Penny Morrell\', \'Simon Fisher-Turner ... Doctor Gilbert\', \'Jason Isaacs\', \'Deborah Findlay\', \'Nicholas Hewetson\', \'Jack McKenzie\', \'Claire Ashton\', \'Jeremy Caleb Johnson\']'